In [1]:
### Declarations for libraries used in this code ###
import csv 
import numpy as np
from scipy.sparse import coo_matrix as coo
from sklearn.decomposition import TruncatedSVD as Tr
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
### Definition for binary utility matrix ###
def binary_utility_matrix(user_ids, item_ids):
    items = set()
    for lst in item_ids: 
        items.update(lst)
    a = {user:item for item,user in enumerate(list(dict.fromkeys(user_ids)))}
    b = {item:i for i,item in enumerate(sorted(items))}
    rows, cols, data = list(), list(), list()
    for user_id, item_id in zip(user_ids, item_ids):
        user = a[user_id]
        for it in item_id:
            rows.append(user) 
            cols.append(b.get(it)) 
            data.append(1.0)
    user_id = coo((data, (rows, cols)), shape=(len(a), len(b)), dtype=np.float32).tocsr()
    user_id.data[:] = 1.0
    user_id.eliminate_zeros()
    return user_id, {item:user for user,item in a.items()}, {i:item for item,i in b.items()}

In [4]:
### Definition about Item-Based kNN algorithm and 20 item recommendation ###
# Compute Item-Item Similarity (the "KNN" step)
def item_based_knn(user_id, k, batch, n_components, n, idx2iid):
    User, Item = user_id.shape
    Dimention = min(n_components, max(2, min(User, Item)-1))
    # SVD projects hugh item vectors (size: 31667) into a compact latent space where similar items stay close -
    # it's still item-based kNN, but using learned embeddings instead of raw co-occurrence vectors.
    # In this code, svd option is already set True. 
    # This speeds up computation, reduces memory/VRAM, and slightly smooths noisy similarities.
    svd = Tr(n_components=Dimention, random_state=42)
    X_np = svd.fit_transform(user_id.T).astype(np.float32, copy=False) 
    rn = np.linalg.norm(X_np, axis=1, keepdims=True)
    rn[rn < 1e-12] = 1.0
    # X_np / rn converts item vectors to unit length.
    # So, their dot product equals cosine similarity.
    Tor = torch.from_numpy(X_np / rn).to(device)                     
    rows, cols, data = list(), list(), list()
    with torch.inference_mode():
        for st in range(0, Tor.shape[0], batch):     
            # Compute similarities on the GPU.   
            # All pairwise cosine similarities between Q (Tor[st:min(Tor.shape[0], st + batch)])'s items and all items.
            # Do it in batches to avoid exceeding GPU memory.        
            sims = Tor[st:min(Tor.shape[0], st + batch)] @ Tor.t().contiguous()                     
            row_idx = torch.arange(min(Tor.shape[0], st + batch) - st, device=device)
            col_idx = torch.arange(st, min(Tor.shape[0], st + batch), device=device)
            sims[row_idx, col_idx] = float('-inf')
            # Keep only the top-k neighbors per item.
            # For each item, we store the k most similar items with the highest cosine similarity.
            # The result is a sparse item-item similarity matrix Sa, where each item has up to k neighbors. 
            vals, idx = torch.topk(sims, k=min(k, max(1, Tor.shape[0]-1)), dim=1, largest=True, sorted=False)
            idx_np = idx.cpu().numpy()
            vals_np = vals.cpu().numpy().astype(np.float32)
            for i in range(min(Tor.shape[0], st + batch) - st):
                gi = st + i
                for j, s in zip(idx_np[i], vals_np[i]):
                    if np.isfinite(s) and s > 0.0:
                        rows.append(gi)
                        cols.append(int(j)) 
                        data.append(float(s))
    Sa = coo((data, (rows, cols)), shape=(Tor.shape[0], Tor.shape[0]), dtype=np.float32).tocsr()
    Sa.setdiag(0.0)
    Sa.eliminate_zeros()
    # Once we have Sa (item similarities), the recommender predicts how much a user u would like each unseen item.
    # user_id = (users x items): what each user has interacted with (1s and 0s).
    # Sa = (items x items): similarity between items.
    scores = (user_id @ Sa).tolil()   
    ui_csr = user_id.tocsr()
    for user in range(ui_csr.shape[0]):
        begin, finish = ui_csr.indptr[user], ui_csr.indptr[user+1]
        for j in ui_csr.indices[begin:finish]:
            scores[user, j] = -np.inf
    scores = scores.tocsr()
    popular = np.argsort(-np.asarray(user_id.sum(axis=0)).ravel())
    recs = dict()
    for user in range(scores.shape[0]):
        good = [(j, s) for j, s in zip(scores.getrow(user).indices, scores.getrow(user).data) if np.isfinite(s)]
        if len(good) > n:
            # After similarity scores for candidate items are obtained,
            # quickly finds the indices of the top-n scores (partial sort).
            top = [good[k] for k in np.argpartition(np.array([g[1] for g in good]), -n)[-n:]]
            top.sort(key=lambda x: -x[1])
            js = [j for j,k in top]
        else:
            good.sort(key=lambda x: -x[1])
            js = [j for j,k in good][:n]
        if len(js) < n:
            seen = set(ui_csr.indices[ui_csr.indptr[user]:ui_csr.indptr[user+1]])
            for p in popular:
                # Users shouldn't be recommended items they've already interacted with.
                if p in seen or p in js: 
                    continue
                js.append(p)
                if len(js) >= n: 
                    break
        # For each user u, we take the Top-N highest-scoring unseen items.
        recs[user] = [idx2iid[j] for j in js[:n]]
    return recs

In [5]:
def main():
    ### Insert dataset ###
    user_ids, item_ids = list(), list()
    with open("train-1.txt", "r", encoding="utf-8") as g:
        for lin in g:
            tokens = [t for t in lin.split() if t is not None]
            user_ids.append(tokens[0])
            item_ids.append(list(dict.fromkeys(tokens[1:])))

    ### Build binary utility matrix ###
    ui, uid, iid = binary_utility_matrix(user_ids, item_ids)

    ### Recommend 20 items per user using item-based kNN ###
    recs = item_based_knn(ui, k=100, batch=200000, n_components=256, n=20, idx2iid=iid)
    
    ### The recommended items are written in recommendations.csv file ###
    with open("recommendations.csv", "w", newline="", encoding="utf-8") as g:
        w = csv.writer(g)
        w.writerow(["user_id", "recommendations"])
        for uidx in range(len(uid)):
            ints = [int(num) for num in recs[uidx]]
            ints.sort()  # Written in ascending order
            w.writerow([uid[uidx], " ".join(map(str, ints))])

In [6]:
if __name__ == "__main__":
    main()